In [12]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Import data
import pandas as pd

FLIGHT_PATH = "/content/drive/My Drive/Capstone Project AA/Flight_Data/AA_FLIGHT_RECORD_PROCESSED/aa_flights_entire.csv" # Original flight & weather data files
WEATHER_PATH = "/content/drive/My Drive/Capstone Project AA/Weather_Data_Hubs/All_Hubs_Weather_EDA.csv" # Date rollover issue solved already

flight_df = pd.read_csv(FLIGHT_PATH, low_memory=False)
weather_df = pd.read_csv(WEATHER_PATH, low_memory=False)

print("flight_df shape :", flight_df.shape)
print("weather_df shape:", weather_df.shape)

print("Flight DataFrame head:")
display(flight_df.head())

print("\nWeather DataFrame head:")
display(weather_df.head())

flight_df shape : (399618, 35)
weather_df shape: (2254603, 10)
Flight DataFrame head:


,FL_DATE,MKT_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,WHEELS_OFF,CRS_ARR_TIME,ARR_TIME,...,ORIGIN_UTC_OFFSET,DEST_UTC_OFFSET,CRS_DEP_TIME_UTC_MIN,DEP_TIME_UTC_MIN,CRS_ARR_TIME_UTC_MIN,ARR_TIME_UTC_MIN,CRS_DEP_TIME_UTC_HHMM,DEP_TIME_UTC_HHMM,CRS_ARR_TIME_UTC_HHMM,ARR_TIME_UTC_HHMM
0,2023-01-01,AA,1,JFK,LAX,730,726.0,743.0,1052,1054.0,...,-5,-8,750,746.0,1132,1134.0,1230,1226,1852,1854
1,2023-01-02,AA,1,JFK,LAX,730,723.0,742.0,1052,1037.0,...,-5,-8,750,743.0,1132,1117.0,1230,1223,1852,1837
2,2023-01-03,AA,1,JFK,LAX,730,807.0,829.0,1052,1140.0,...,-5,-8,750,787.0,1132,1180.0,1230,1307,1852,1940
3,2023-01-04,AA,1,JFK,LAX,730,727.0,747.0,1052,1039.0,...,-5,-8,750,747.0,1132,1119.0,1230,1227,1852,1839
4,2023-01-05,AA,1,JFK,LAX,730,725.0,746.0,1052,1046.0,...,-5,-8,750,745.0,1132,1126.0,1230,1225,1852,1846



Weather DataFrame head:


,station,time,temp_C,dewpoint_C,wind_dir_deg,wind_speed_kt,visibility_mi,pressure_mb,weather,Sky
0,KDFW,2023-01-01 00:00:00,18.0,7.0,180.0,8.0,10.0,1008.130685,NaN,"[('CLR', None, None)]"
1,KDFW,2023-01-01 00:00:00,18.3,6.7,180.0,7.0,NaN,NaN,NaN,NaN
2,KDFW,2023-01-01 00:05:00,18.0,7.0,180.0,8.0,10.0,1007.792045,NaN,"[('CLR', None, None)]"
3,KDFW,2023-01-01 00:10:00,19.0,7.0,180.0,8.0,10.0,1008.130685,NaN,"[('CLR', None, None)]"
4,KDFW,2023-01-01 00:15:00,19.0,7.0,180.0,7.0,10.0,1008.130685,NaN,"[('CLR', None, None)]"


In [14]:
# Purpose: create CRS_DEP_DATETIME / CRS_ARR_DATETIME (Not used in classification model for now)

import pandas as pd
import numpy as np

flight_df['FL_DATE'] = pd.to_datetime(flight_df['FL_DATE'], errors='coerce')

# convert numeric time like 730 -> "0730"
for col in ['CRS_DEP_TIME', 'CRS_ARR_TIME']:
    flight_df[col] = flight_df[col].astype(str).str.zfill(4)

# build datetime by string concatenation (much faster)
flight_df['CRS_DEP_DATETIME'] = pd.to_datetime(
    flight_df['FL_DATE'].dt.strftime("%Y-%m-%d") + " " +
    flight_df['CRS_DEP_TIME'].str.slice(0, 2) + ":" +
    flight_df['CRS_DEP_TIME'].str.slice(2, 4),
    errors='coerce'
)

flight_df['CRS_ARR_DATETIME'] = pd.to_datetime(
    flight_df['FL_DATE'].dt.strftime("%Y-%m-%d") + " " +
    flight_df['CRS_ARR_TIME'].str.slice(0, 2) + ":" +
    flight_df['CRS_ARR_TIME'].str.slice(2, 4),
    errors='coerce'
)

print("Built CRS_DEP_DATETIME and CRS_ARR_DATETIME")
display(flight_df[['FL_DATE', 'CRS_DEP_TIME', 'CRS_DEP_DATETIME',
                   'CRS_ARR_TIME', 'CRS_ARR_DATETIME']].head())
print("\nData types:")
print(flight_df[['CRS_DEP_DATETIME', 'CRS_ARR_DATETIME']].dtypes)


Built CRS_DEP_DATETIME and CRS_ARR_DATETIME


,FL_DATE,CRS_DEP_TIME,CRS_DEP_DATETIME,CRS_ARR_TIME,CRS_ARR_DATETIME
0,2023-01-01,0730,2023-01-01 07:30:00,1052,2023-01-01 10:52:00
1,2023-01-02,0730,2023-01-02 07:30:00,1052,2023-01-02 10:52:00
2,2023-01-03,0730,2023-01-03 07:30:00,1052,2023-01-03 10:52:00
3,2023-01-04,0730,2023-01-04 07:30:00,1052,2023-01-04 10:52:00
4,2023-01-05,0730,2023-01-05 07:30:00,1052,2023-01-05 10:52:00



Data types:
CRS_DEP_DATETIME    datetime64[ns]
CRS_ARR_DATETIME    datetime64[ns]
dtype: object


In [15]:
# Clean time columns (convert to int first, then to zero-padded string)
# Not used in classification for now
for col in ['DEP_TIME', 'ARR_TIME']:
    flight_df[col] = (
        flight_df[col]
        .fillna(0)                 # avoid NaN errors
        .astype(float)
        .astype(int)
        .astype(str)
        .str.zfill(4)
    )

# Build datetime columns
flight_df['DEP_DATETIME'] = pd.to_datetime(
    flight_df['FL_DATE'].dt.strftime("%Y-%m-%d") + " " +
    flight_df['DEP_TIME'].str.slice(0, 2) + ":" +
    flight_df['DEP_TIME'].str.slice(2, 4),
    format="%Y-%m-%d %H:%M",
    errors='coerce'
)

flight_df['ARR_DATETIME'] = pd.to_datetime(
    flight_df['FL_DATE'].dt.strftime("%Y-%m-%d") + " " +
    flight_df['ARR_TIME'].str.slice(0, 2) + ":" +
    flight_df['ARR_TIME'].str.slice(2, 4),
    format="%Y-%m-%d %H:%M",
    errors='coerce'
)

print("built DEP_DATETIME and ARR_DATETIME successfully")
display(flight_df[['FL_DATE', 'DEP_TIME', 'DEP_DATETIME', 'ARR_TIME', 'ARR_DATETIME']].head())
print("\nData types:")
print(flight_df[['DEP_DATETIME', 'ARR_DATETIME']].dtypes)


built DEP_DATETIME and ARR_DATETIME successfully


,FL_DATE,DEP_TIME,DEP_DATETIME,ARR_TIME,ARR_DATETIME
0,2023-01-01,0726,2023-01-01 07:26:00,1054,2023-01-01 10:54:00
1,2023-01-02,0723,2023-01-02 07:23:00,1037,2023-01-02 10:37:00
2,2023-01-03,0807,2023-01-03 08:07:00,1140,2023-01-03 11:40:00
3,2023-01-04,0727,2023-01-04 07:27:00,1039,2023-01-04 10:39:00
4,2023-01-05,0725,2023-01-05 07:25:00,1046,2023-01-05 10:46:00



Data types:
DEP_DATETIME    datetime64[ns]
ARR_DATETIME    datetime64[ns]
dtype: object


In [19]:
# Compute delay variables and classification labels ===
# Purpose:
#   1. Calculate departure delay, arrival delay, and in-air delay change (delta delay)
#   2. Create both human-readable (string) and numeric labels for later modeling


# --- 1️.Compute delay durations in minutes ---
# Calculate scheduled vs. actual time difference for departure/arrival
flight_df['dep_delay_calc'] = (
    (flight_df['DEP_DATETIME'] - flight_df['CRS_DEP_DATETIME']).dt.total_seconds() / 60
)
flight_df['arr_delay_calc'] = (
    (flight_df['ARR_DATETIME'] - flight_df['CRS_ARR_DATETIME']).dt.total_seconds() / 60
)

# --- 2️.Compute delta delay ---
# Positive → delay worsened; Negative → delay improved (absorbed)
flight_df['delta_delay_calc'] = (
    flight_df['arr_delay_calc'] - flight_df['dep_delay_calc']
)

# --- 3️.Create categorical label (string form) ---
# improved / neutral / worsened for interpretability
# Drop the original, deprecated delay flag variable first
flight_df = flight_df.drop(columns=['delay_change_flag'])

def classify_delay(x):
    if pd.isna(x):
        return np.nan
    elif x < -5:
        return 'improved'   # improved by more than 5 minutes
    elif -5 <= x <= 5:
        return 'neutral'    # roughly unchanged
    else:
        return 'worsened'   # worsened by more than 5 minutes

flight_df['delay_change_label'] = flight_df['delta_delay_calc'].apply(classify_delay)

# --- 4️.Numeric label for machine learning ---
# Tri-labeling not used in classification model for now, as we mainly discuss whether improve or not--bi-classification.
# Libin's log regression model already includes a 0-1 classification part which is workable.
# Map text labels into integer codes
label_map = {'improved': -1, 'neutral': 0, 'worsened': 1}
flight_df['delay_label_num'] = flight_df['delay_change_label'].map(label_map)

# --- 5️.Display sample results ---
print("Computed delay variables and dual labels (string + numeric) successfully")
display(
    flight_df[
        ['FL_DATE', 'DEP_DATETIME', 'ARR_DATETIME',
         'dep_delay_calc', 'arr_delay_calc', 'delta_delay_calc',
         'delay_change_label', 'delay_label_num']
    ].head(10)
)


Computed delay variables and dual labels (string + numeric) successfully


,FL_DATE,DEP_DATETIME,ARR_DATETIME,dep_delay_calc,arr_delay_calc,delta_delay_calc,delay_change_label,delay_label_num
0,2023-01-01,2023-01-01 07:26:00,2023-01-01 10:54:00,-4.0,2.0,6.0,worsened,1.0
1,2023-01-02,2023-01-02 07:23:00,2023-01-02 10:37:00,-7.0,-15.0,-8.0,improved,-1.0
2,2023-01-03,2023-01-03 08:07:00,2023-01-03 11:40:00,37.0,48.0,11.0,worsened,1.0
3,2023-01-04,2023-01-04 07:27:00,2023-01-04 10:39:00,-3.0,-13.0,-10.0,improved,-1.0
4,2023-01-05,2023-01-05 07:25:00,2023-01-05 10:46:00,-5.0,-6.0,-1.0,neutral,0.0
5,2023-01-06,2023-01-06 07:23:00,2023-01-06 10:23:00,-7.0,-29.0,-22.0,improved,-1.0
6,2023-01-07,2023-01-07 07:24:00,2023-01-07 10:29:00,-6.0,-23.0,-17.0,improved,-1.0
7,2023-01-08,2023-01-08 07:23:00,2023-01-08 10:26:00,-7.0,-26.0,-19.0,improved,-1.0
8,2023-01-09,2023-01-09 07:26:00,2023-01-09 10:33:00,-4.0,-19.0,-15.0,improved,-1.0
9,2023-01-10,2023-01-10 07:21:00,2023-01-10 10:53:00,-8.0,-1.0,7.0,worsened,1.0


In [20]:
# === Prepare weather data for alignment ===
# Purpose: Convert weather times, round them, and extract airport codes safely.

import pandas as pd

# 1️. Make sure we are operating on the DataFrame object
assert isinstance(weather_df, pd.DataFrame), "weather_df must be a pandas DataFrame"

# 2️. Convert 'time' column to datetime (ignore parse errors)
weather_df['time'] = pd.to_datetime(weather_df['time'], errors='coerce')

# 3️. Round to nearest 5 minutes
weather_df['time_rounded'] = weather_df['time'].dt.round('5min')

# 4️. Extract airport code (last 3 letters of 'station')
weather_df['airport_code'] = weather_df['station'].astype(str).str[-3:]

# 5️. Display preview
print("Weather data processed successfully (time converted, rounded, airport code extracted).")
display(weather_df[['station', 'airport_code', 'time', 'time_rounded']].head(10))


Weather data processed successfully (time converted, rounded, airport code extracted).


,station,airport_code,time,time_rounded
0,KDFW,DFW,2023-01-01 00:00:00,2023-01-01 00:00:00
1,KDFW,DFW,2023-01-01 00:00:00,2023-01-01 00:00:00
2,KDFW,DFW,2023-01-01 00:05:00,2023-01-01 00:05:00
3,KDFW,DFW,2023-01-01 00:10:00,2023-01-01 00:10:00
4,KDFW,DFW,2023-01-01 00:15:00,2023-01-01 00:15:00
5,KDFW,DFW,2023-01-01 00:20:00,2023-01-01 00:20:00
6,KDFW,DFW,2023-01-01 00:25:00,2023-01-01 00:25:00
7,KDFW,DFW,2023-01-01 00:30:00,2023-01-01 00:30:00
8,KDFW,DFW,2023-01-01 00:35:00,2023-01-01 00:35:00
9,KDFW,DFW,2023-01-01 00:40:00,2023-01-01 00:40:00


In [22]:
# === Step 3B: Match weather records 1 hour before departure & arrival ===
# Purpose:
#   1. For each flight, find weather observations ~1 hour before its scheduled
#      departure and arrival at the corresponding airports.
#   2. Append these weather features into the flight dataframe.

from pandas import Timedelta

# 1. Create reference times for weather lookup
# Scheduled departure/arrival minus 1 hour (makes sense since scheduled times are known prior to departure)
flight_df['DEP_WEATHER_TIME'] = flight_df['CRS_DEP_DATETIME'] - Timedelta(hours=1)
flight_df['ARR_WEATHER_TIME'] = flight_df['CRS_ARR_DATETIME'] - Timedelta(hours=1)

# 2. Ensure weather data is sorted by time for merge_asof
weather_sorted = weather_df.sort_values('time_rounded')

# 3. Merge departure-weather (ORIGIN airport)
merged_dep = pd.merge_asof(
    flight_df.sort_values('DEP_WEATHER_TIME'),
    weather_sorted,
    left_on='DEP_WEATHER_TIME',
    right_on='time_rounded',
    left_by='ORIGIN',
    right_by='airport_code',
    direction='nearest',
    tolerance=Timedelta('1h')   # allow up to 1 hour difference
)

# Rename columns with DEP_ prefix
dep_cols = ['temp_C', 'dewpoint_C', 'wind_dir_deg', 'wind_speed_kt',
            'visibility_mi', 'pressure_mb', 'weather', 'Sky']
merged_dep = merged_dep.rename(columns={col: f'DEP_{col}' for col in dep_cols})

# 4. Merge arrival-weather (DEST airport)
merged_dep_sorted = merged_dep.sort_values('ARR_WEATHER_TIME')
merged_final = pd.merge_asof(
    merged_dep_sorted,
    weather_sorted,
    left_on='ARR_WEATHER_TIME',
    right_on='time_rounded',
    left_by='DEST',
    right_by='airport_code',
    direction='nearest',
    tolerance=Timedelta('1h')
)

# Rename arrival columns with ARR_ prefix
arr_cols = ['temp_C', 'dewpoint_C', 'wind_dir_deg', 'wind_speed_kt',
            'visibility_mi', 'pressure_mb', 'weather', 'Sky']
merged_final = merged_final.rename(columns={col: f'ARR_{col}' for col in arr_cols})

# 5. Clean up redundant columns
merged_final = merged_final.drop(columns=['station', 'airport_code', 'time', 'time_rounded'],
                                 errors='ignore')

print("Successfully matched weather records (1 hour before dep/arr).")
display(merged_final.head(10))


Successfully matched weather records (1 hour before dep/arr).


,FL_DATE,MKT_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,WHEELS_OFF,CRS_ARR_TIME,ARR_TIME,...,ARR_temp_C,ARR_dewpoint_C,ARR_wind_dir_deg,ARR_wind_speed_kt,ARR_visibility_mi,ARR_pressure_mb,ARR_weather,ARR_Sky,time_rounded_y,airport_code_y
0,2023-01-01,AA,601,PHX,ORD,1945,2029,2044.0,0013,0029,...,3.0,-3.0,0.0,0.0,10.0,1011.855722,NaN,"[('BKN', <metar.Datatypes.distance object at 0...",2023-01-01 00:00:00,ORD
1,2023-01-01,AA,522,CLT,DFW,2220,2217,2231.0,0016,2352,...,18.0,7.0,180.0,8.0,10.0,1008.130685,NaN,"[('CLR', None, None)]",2023-01-01 00:00:00,DFW
2,2023-01-01,AA,185,JFK,LAX,2030,2024,2047.0,0016,0008,...,15.0,14.0,210.0,7.0,10.0,1013.548921,NaN,"[('SCT', <metar.Datatypes.distance object at 0...",2023-01-01 00:00:00,LAX
3,2023-01-01,AA,2468,MIA,LAX,2135,2141,2157.0,0022,0016,...,15.0,14.0,210.0,7.0,10.0,1013.548921,NaN,"[('SCT', <metar.Datatypes.distance object at 0...",2023-01-01 00:00:00,LAX
4,2023-01-01,AA,1091,CLT,PHL,2240,2240,2257.0,0023,0007,...,12.0,12.0,240.0,5.0,1.5,1010.839803,"[('', None, 'RA', None, None), ('', None, '', ...","[('OVC', <metar.Datatypes.distance object at 0...",2023-01-01 00:00:00,PHL
5,2023-01-01,AA,695,DFW,MIA,2042,2048,2101.0,0024,0018,...,24.0,21.0,0.0,0.0,10.0,1020.321717,NaN,"[('FEW', <metar.Datatypes.distance object at 0...",2023-01-01 00:00:00,MIA
6,2023-01-01,AA,2222,DFW,DCA,2044,2051,2106.0,0028,0028,...,13.0,13.0,210.0,7.0,4.0,1011.855722,"[('-', None, 'RA', None, None), ('', None, '',...","[('BKN', <metar.Datatypes.distance object at 0...",2023-01-01 00:00:00,DCA
7,2023-01-01,AA,1277,ORD,MIA,2027,2045,2100.0,0039,0030,...,24.0,21.0,0.0,0.0,10.0,1020.321717,NaN,"[('FEW', <metar.Datatypes.distance object at 0...",2023-01-01 00:00:00,MIA
8,2023-01-01,AA,994,CLT,MIA,2230,2339,2356.0,0041,0128,...,24.0,21.0,0.0,0.0,10.0,1020.321717,NaN,"[('FEW', <metar.Datatypes.distance object at 0...",2023-01-01 00:00:00,MIA
9,2023-01-01,AA,483,LAX,DFW,2359,0050,105.0,0500,0542,...,16.0,7.0,180.0,8.0,10.0,1009.485244,NaN,"[('CLR', None, None)]",2023-01-01 04:00:00,DFW


In [23]:

# === Step 3C: Append direction data to flights ===
# Purpose:
#   1. For each flight, derive rough direction data (text) from lat/lon information of airports
#   2. Append these features into the flight dataframe.import pandas as pd

# Create a dictionary with the airport data
airport_data = {
    'airport_code': ['DFW', 'CLT', 'LAX', 'ORD', 'MIA', 'PHL', 'JFK', 'DCA', 'PHX', 'LGA'],
    'latitude': [32.8998, 35.2140, 33.9416, 41.9742, 25.7959, 39.8719, 40.6397, 38.8522, 33.4342, 40.7750],
    'longitude': [-97.0403, -80.9431, -118.4085, -87.9073, -80.2870, -75.2411, -73.7789, -77.0378, -113.0117, -73.8750]
}

# Create the DataFrame
airport_coords = pd.DataFrame(airport_data)



In [25]:
# This block consolidates the merging of airport coordinates and calculation of flight direction.

# Import necessary libraries
import pandas as pd
import numpy as np
import math # Import math for trigonometric calculations

# --- Defensive Cleanup ---
# Drop any potentially lingering coordinate columns from previous failed merges or executions.
# This ensures a clean state for the merge operations that follow.
cols_to_drop_if_exist = [
    'latitude', 'longitude', # Base names if they were merged directly
    'latitude_x', 'longitude_x', 'latitude_y', 'longitude_y', # Default pandas suffixes for latitude/longitude
    'ORIGIN_LAT', 'ORIGIN_LON', # If these already existed from a prior run
    'DEST_LAT', 'DEST_LON',
    'ORIGIN_LAT_x', 'ORIGIN_LON_x', 'ORIGIN_LAT_y', 'ORIGIN_LON_y', # Suffixes from prior attempts on ORIGIN
    'DEST_LAT_x', 'DEST_LON_x', 'DEST_LAT_y', 'DEST_LON_y',     # Suffixes from prior attempts on DEST
    'airport_code', 'airport_code_x', 'airport_code_y' # Redundant merge keys
]
# Filter out columns that don't exist in merged_final to prevent error if they aren't there
existing_cols_to_drop = [col for col in cols_to_drop_if_exist if col in merged_final.columns]
if existing_cols_to_drop:
    merged_final = merged_final.drop(columns=existing_cols_to_drop, errors='ignore')
print("Cleaned up potential conflicting columns from merged_final.")


# --- Merge Origin Coordinates ---
# Merge the longitude and latitude for the departure (ORIGIN) airports into the `merged_final` DataFrame.
# The `ORIGIN` column in `merged_final` is matched with the `airport_code` in `airport_coords`.
# New columns `ORIGIN_LAT` and `ORIGIN_LON` are created.
airport_coords_origin_renamed = airport_coords[['airport_code', 'latitude', 'longitude']].copy()
# Rename columns in the right DataFrame BEFORE merging to avoid conflicts with 'latitude', 'longitude'
# if they somehow exist in 'merged_final' or are generated by pandas' default suffixing.
airport_coords_origin_renamed = airport_coords_origin_renamed.rename(columns={
    'airport_code': 'origin_airport_code', # Make merge key unique
    'latitude': 'ORIGIN_LAT_TEMP',        # Temporary unique name
    'longitude': 'ORIGIN_LON_TEMP'         # Temporary unique name
})

merged_final = pd.merge(
    merged_final,
    airport_coords_origin_renamed,
    left_on='ORIGIN',
    right_on='origin_airport_code', # Use unique merge key
    how='left'
)
# Drop the redundant unique 'origin_airport_code' column introduced by the merge
merged_final = merged_final.drop(columns=['origin_airport_code'], errors='ignore')
# Rename the temporary columns to their final desired names
merged_final = merged_final.rename(columns={
    'ORIGIN_LAT_TEMP': 'ORIGIN_LAT',
    'ORIGIN_LON_TEMP': 'ORIGIN_LON'
})
print("Merged origin coordinates successfully.")

# --- Merge Destination Coordinates ---
# Merge the longitude and latitude for the arrival (DEST) airports into the `merged_final` DataFrame.
# The `DEST` column in `merged_final` is matched with the `airport_code` in `airport_coords`.
# New columns `DEST_LAT` and `DEST_LON` are created.
airport_coords_dest_renamed = airport_coords[['airport_code', 'latitude', 'longitude']].copy()
# Rename columns in the right DataFrame BEFORE merging to avoid conflicts
airport_coords_dest_renamed = airport_coords_dest_renamed.rename(columns={
    'airport_code': 'dest_airport_code', # Make merge key unique
    'latitude': 'DEST_LAT_TEMP',         # Temporary unique name
    'longitude': 'DEST_LON_TEMP'          # Temporary unique name
})

merged_final = pd.merge(
    merged_final,
    airport_coords_dest_renamed,
    left_on='DEST',
    right_on='dest_airport_code', # Use unique merge key
    how='left'
)
# Drop the redundant unique 'dest_airport_code' column introduced by the merge
merged_final = merged_final.drop(columns=['dest_airport_code'], errors='ignore')
# Rename the temporary columns to their final desired names
merged_final = merged_final.rename(columns={
    'DEST_LAT_TEMP': 'DEST_LAT',
    'DEST_LON_TEMP': 'DEST_LON'
})
print("Merged destination coordinates successfully.")

# --- Calculate Flight Direction ---
# Define a function to determine the cardinal flight direction based on origin and destination coordinates.
# A small threshold is used to distinguish significant movement.
def get_flight_direction(origin_lat, origin_lon, dest_lat, dest_lon):
    if pd.isna(origin_lat) or pd.isna(origin_lon) or pd.isna(dest_lat) or pd.isna(dest_lon):
        return np.nan # Return NaN if any coordinate is missing

    lat_diff = dest_lat - origin_lat
    lon_diff = dest_lon - origin_lon

    direction = []
    threshold = 0.001 # A small threshold for significant movement

    if lat_diff > threshold:
        direction.append('North')
    elif lat_diff < -threshold:
        direction.append('South')

    if lon_diff > threshold:
        direction.append('East')
    elif lon_diff < -threshold:
        direction.append('West')

    if not direction:
        return 'Local/Same Location' # Categorize as 'Local/Same Location' if no significant movement
    elif len(direction) == 2:
        return ''.join(direction) # e.g., 'NorthEast', 'SouthWest'
    else:
        return direction[0] # e.g., 'North', 'East'

# Apply the function to create the new 'FLIGHT_DIRECTION' column in merged_final
merged_final['FLIGHT_DIRECTION'] = merged_final.apply(
    lambda row: get_flight_direction(
        row['ORIGIN_LAT'], row['ORIGIN_LON'], row['DEST_LAT'], row['DEST_LON']
    ), axis=1
)
print("Flight direction calculated successfully.")

# --- Calculate Flight Direction Vector (Bearing) ---
# Define a function to calculate the flight direction vector (bearing) in degrees [0, 360).
# 0 = North, 90 = East, 180 = South, 270 = West.
def get_flight_direction_vector(origin_lat, origin_lon, dest_lat, dest_lon):
    if pd.isna(origin_lat) or pd.isna(origin_lon) or pd.isna(dest_lat) or pd.isna(dest_lon):
        return np.nan

    # Convert latitudes and longitudes from degrees to radians
    lat1_rad = math.radians(origin_lat)
    lon1_rad = math.radians(origin_lon)
    lat2_rad = math.radians(dest_lat)
    lon2_rad = math.radians(dest_lon)

    delta_lon = lon2_rad - lon1_rad

    y = math.sin(delta_lon) * math.cos(lat2_rad)
    x = math.cos(lat1_rad) * math.sin(lat2_rad) - math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(delta_lon)

    # Calculate bearing in radians, then convert to degrees
    bearing_rad = math.atan2(y, x)
    bearing_deg = math.degrees(bearing_rad)

    # Normalize bearing to be in the range [0, 360) and round to the nearest integer
    # The +360 and %360 ensures positive values and correct modulo operation for negative angles.
    bearing_deg = (bearing_deg + 360) % 360

    return round(bearing_deg)

# Apply the function to create the new 'FLIGHT_DIRECTION_VECTOR' column
merged_final['FLIGHT_DIRECTION_VECTOR'] = merged_final.apply(
    lambda row: get_flight_direction_vector(
        row['ORIGIN_LAT'], row['ORIGIN_LON'], row['DEST_LAT'], row['DEST_LON']
    ), axis=1
)
print("Flight direction vector calculated successfully.")

# --- Display Sample Results ---
# Display the head of the DataFrame with the newly added coordinate and direction columns
print("\nPreview of merged_final with newly added coordinate and direction columns:")
#display(merged_final[['ORIGIN', 'DEST', 'ORIGIN_LAT', 'ORIGIN_LON', 'DEST_LAT', 'DEST_LON', 'FLIGHT_DIRECTION']].head())
display(merged_final.head())

Cleaned up potential conflicting columns from merged_final.
Merged origin coordinates successfully.
Merged destination coordinates successfully.
Flight direction calculated successfully.
Flight direction vector calculated successfully.

Preview of merged_final with newly added coordinate and direction columns:


,FL_DATE,MKT_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,WHEELS_OFF,CRS_ARR_TIME,ARR_TIME,...,ARR_pressure_mb,ARR_weather,ARR_Sky,time_rounded_y,FLIGHT_DIRECTION,ORIGIN_LAT,ORIGIN_LON,DEST_LAT,DEST_LON,FLIGHT_DIRECTION_VECTOR
0,2023-01-01,AA,601,PHX,ORD,1945,2029,2044.0,0013,0029,...,1011.855722,NaN,"[('BKN', <metar.Datatypes.distance object at 0...",2023-01-01,NorthEast,33.4342,-113.0117,41.9742,-87.9073,59
1,2023-01-01,AA,522,CLT,DFW,2220,2217,2231.0,0016,2352,...,1008.130685,NaN,"[('CLR', None, None)]",2023-01-01,SouthWest,35.2140,-80.9431,32.8998,-97.0403,265
2,2023-01-01,AA,185,JFK,LAX,2030,2024,2047.0,0016,0008,...,1013.548921,NaN,"[('SCT', <metar.Datatypes.distance object at 0...",2023-01-01,SouthWest,40.6397,-73.7789,33.9416,-118.4085,274
3,2023-01-01,AA,2468,MIA,LAX,2135,2141,2157.0,0022,0016,...,1013.548921,NaN,"[('SCT', <metar.Datatypes.distance object at 0...",2023-01-01,NorthWest,25.7959,-80.2870,33.9416,-118.4085,293
4,2023-01-01,AA,1091,CLT,PHL,2240,2240,2257.0,0023,0007,...,1010.839803,"[('', None, 'RA', None, None), ('', None, '', ...","[('OVC', <metar.Datatypes.distance object at 0...",2023-01-01,NorthEast,35.2140,-80.9431,39.8719,-75.2411,42


In [26]:
# === Step 4: Export Clean Model-Ready Data ===
# Purpose:
#   1. Include only scheduled departure/arrival times and convert them into hours
#      to avoid over-granularity.
#   2. Create two datasets:
#        (a) model_input.csv  -> For model training and prediction (no actual times)
#        (b) model_eval.csv   -> For model evaluation (includes actual times)
#   3. Save both files to Google Drive under /Modeling/merged_data.

import pandas as pd
import os

# Define export path
EXPORT_DIR = "/content/drive/My Drive/Capstone Project AA/Modeling/merged_data_deploy" # Final data folder to be used in both regression and classification
os.makedirs(EXPORT_DIR, exist_ok=True)

# 1. Convert scheduled times to numeric safely
# These columns are in HHMM integer/string format (e.g., 730 -> 7:30 AM)
merged_final['CRS_DEP_TIME'] = pd.to_numeric(merged_final['CRS_DEP_TIME'], errors='coerce')
merged_final['CRS_ARR_TIME'] = pd.to_numeric(merged_final['CRS_ARR_TIME'], errors='coerce')

# 2. Derive hour-based features (avoid over-granularity)
merged_final['CRS_DEP_HOUR'] = (merged_final['CRS_DEP_TIME'] // 100).astype('Int64')
merged_final['CRS_ARR_HOUR'] = (merged_final['CRS_ARR_TIME'] // 100).astype('Int64')

print("Converted CRS_DEP_TIME and CRS_ARR_TIME to numeric, then derived hour-based features.\n")

# 3. Define actual-time columns to remove from model input (prevent label leakage)
actual_cols = [
    'DEP_TIME', 'ARR_TIME', 'WHEELS_OFF', 'WHEELS_ON',
    'DEP_DATETIME', 'ARR_DATETIME'
]

# 4. Dataset for model training/prediction (no actual times)
# As for now, no need to separately process a training set since we will be selecting features and doing train-test split in the modeling part
'''
model_input = merged_final.drop(columns=actual_cols, errors='ignore')
input_path = os.path.join(EXPORT_DIR, "model_input.csv")
model_input.to_csv(input_path, index=False)
print(f"Model input dataset exported to: {input_path}")
print(f"Shape: {model_input.shape[0]} rows × {model_input.shape[1]} columns\n")
'''

# 5. Dataset for model evaluation (keep all columns)
# Save as original dataset
#model_eval = merged_final.copy()
#eval_path = os.path.join(EXPORT_DIR, "model_eval.csv")
model_final = merged_final.copy()
#model_eval.to_csv(eval_path, index=False)
final_path = os.path.join(EXPORT_DIR, "model_final.csv")
model_final.to_csv(final_path, index=False)
print(f"Model evaluation dataset exported to: {final_path}")
print(f"Shape: {model_final.shape[0]} rows × {model_final.shape[1]} columns\n")

# 6. Quick verification of exported structure
cols_show = [
    'FL_DATE', 'ORIGIN', 'DEST',
    'CRS_DEP_HOUR', 'CRS_ARR_HOUR',
    'dep_delay', 'arr_delay', 'delta_delay',
    'DEP_temp_C', 'ARR_temp_C'
]

#print("Preview of exported model_input data:")
#display(model_input[cols_show].head(10))

print("Step 4 completed successfully.")

Converted CRS_DEP_TIME and CRS_ARR_TIME to numeric, then derived hour-based features.

Model evaluation dataset exported to: /content/drive/My Drive/Capstone Project AA/Modeling/merged_data_deploy/model_final.csv
Shape: 399618 rows × 75 columns

Step 4 completed successfully.
